In [ ]:
from policyengine_us import Microsimulation
import pandas as pd

In [ ]:
sim = Microsimulation()

In [ ]:
df = sim.calculate("household_net_income". period = 2024)

In [ ]:
df = sim.calculate_dataframe(["household_market_income", "household_net_income", "household_size", "tax_unit_dependents", "household_weight"], period = 2024)

In [ ]:
type(df)
help(microdf.generic.MicroDataFrame)

In [ ]:
print(df['household_net_income'])

In [ ]:
# df_1 = pd.DataFrame(df)
# df_1.columns

In [ ]:
df_2 = df.sample(n = int(sum(df["household_weight"])), 
                 weights = df["household_weight"], 
                 random_state = 1,
                 replace = True
                 )

In [ ]:
if df_2.shape[0] % 2 == 0:
    df_2 = df_2
else:
    df_2 = df_2.drop(df_2.index[-1])

In [ ]:
df_2.shape[0]

In [ ]:
# Split dataframe

# // is the floor division operator
half_index = df_2.shape[0] // 2

df_2 = df_2.reset_index(drop=True)

df_2 = pd.concat([
    # .iloc[:x] selects all rows from the beginning up to, but not including, the row at position x
        df_2.iloc[:half_index], 
        df_2.iloc[half_index:]
    ], axis=1)

In [ ]:
import numpy as np

df_2.shape[1]
df_2.columns

new_column_names = ["y", "y_minus_T", "A", "A_minus_a", "w"]

column_names_j = []
for j in new_column_names:
    new_name = str(j) + "_j"
    column_names_j.append(new_name)

column_names_i = []
for i in new_column_names:
    new_name = str(i) + "_i"
    column_names_i.append(new_name)

df_2.columns = column_names_i + column_names_j

df_2['I'] = np.where(
    (df_2['y_i'] - df_2['y_minus_T_i']) > (df_2['y_j'] - df_2['y_minus_T_j']),
    1, 0
    )

In [ ]:
df_2['']

In [ ]:
import numpy as np
import scipy.optimize as optim
from scipy.stats import norm
from numpy import log

# def neg_log_likelihood(params, data):
#       tau, theta, sigma = params
#       cdf = norm.cdf(
#           (tau * np.log(data['y_i']/data['y_j'])
#            + np.log((data['A_i'] - data['A_minus_a_i']) / (data['A_j'] - data['A_minus_a_j']))
#            + theta * np.log(data['A_i'] / data['A_j'])
#           )/ (np.power(2, 1/2)*sigma)
#       )
#       log_likelihood = np.sum(
#             data['I'] * np.log(cdf) + (1 - data['I']) * np.log(1 - cdf)
#             )
#       return -log_likelihood

# norm.cdf()

# initial_params = [0.6, 1, 5]

# result = optim.minimize(neg_log_likelihood, initial_params, args = (df_2,))

# def neg_log_likelihood(params, data):
#       tau, theta, sigma = params
#       y_i, y_j = data['y_i'], data['y_j']
#       A_i, A_j = data['A_i'], data['A_j']
#       a_i = A_i - data['A_minus_a_i']
#       a_j = A_j - data['A_minus_a_j']
#       I = data['I']

#       cdf = norm.cdf(
#             (tau * np.log(y_i / y_j)
#             + np.log(a_i / a_j)
#             + theta * np.log(A_i / A_j)
#             ),
#             scale = pow(2, 1/2) * sigma
#       )
#       log_likelihood = np.sum(
#             I * np.log(cdf) + (1 - I) * np.log(1 - cdf)
#             )
#       return -log_likelihood

# initial_params = [0.6, 1, 5]

# result = optim.minimize(neg_log_likelihood, initial_params, args = (df_2,))

In [ ]:
def neg_log_likelihood(params, data):
      tau, theta, sigma = params
      y_i, y_j = data['y_i'], data['y_j']
      A_i, A_j = data['A_i'], data['A_j']
      a_i = A_i - data['A_minus_a_i']
      a_j = A_j - data['A_minus_a_j']
      I = data['I']

      # Calculate the utility function components
      utility = (tau * np.log(y_i / y_j)
            + np.log(a_i / a_j)
            + theta * np.log(A_i / A_j))

      # Correct way to use norm.cdf with scale parameter
      scaled_utility = utility / (np.sqrt(2) * sigma)
      cdf = norm.cdf(scaled_utility)

      # Handle potential numerical issues with log(0) or log(1)
      cdf = np.clip(cdf, 1e-10, 1-1e-10)

      log_likelihood = np.sum(
            I * np.log(cdf) + (1 - I) * np.log(1 - cdf)
            )
      return -log_likelihood

initial_params = [0.6, 1, 5]

result = optim.minimize(neg_log_likelihood, initial_params, args = (df_2,))

In [ ]:
result.x